# Two-point microrheology (example)

Load a CSV of tracer trajectories and compute two-point displacement correlations + derived MSDs using `trajkit.flow.two_point`.

This mirrors the classic Crocker/Mason MATLAB workflow (`twopoint.m` → `msdd`).

Download a prepared Parquet bundle from Hugging Face and load it into a TrajectorySet (fastest path).

In [ ]:
import numpy as np
import pandas as pd
from trajkit.flow.two_point import (
    compute_two_point_correlation,
    distinct_msd_from_two_point,
    compute_shear_modulus_from_msd,
)
from trajkit.traj.core import TrajectorySet

from pathlib import Path
from huggingface_hub import snapshot_download
from trajkit import load_trajectory_set


subpath = "data/parquet/experiment_001_2017-08-16/exp001_t027m_r01um_2017-08-16"

local_root = snapshot_download(
    repo_id="m-aban/air-water",
    repo_type="dataset",
    allow_patterns=[f"{subpath}/*"],
    local_dir="hf_cache",
    local_dir_use_symlinks=False,
)

folder = Path(local_root) / subpath
ts = load_trajectory_set(folder)        


Compute two-point displacement correlations using log-spaced lags.

In [ ]:
# Compute two-point correlations
tau = np.ceil(np.logspace(0,2,10)) # lag time
corr = compute_two_point_correlation(
    ts,
    track_id_col="id",
    time_col="t",
    position_cols=("x", "y"),
    dt_values=tau,      # auto log-spaced lags
    max_dt=100,          # max lag in frames if dt_values=None
    r_min=20,
    r_max=500,
    n_r_bins=20,
    clip_to_shared_frames=True,
)


In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from importlib.resources import files

cmap = plt.get_cmap("viridis")
n = len(corr.dt)

plt.figure(figsize=(7, 5))
for i in range(n):
    y = corr.longitudinal[i]
    mask = np.isfinite(y)
    plt.scatter(
        corr.r[mask],
        y[mask],
        s=20,
        color=cmap(i / max(n - 1, 1)),
        label=f"dt={corr.dt[i]:.2g} s",
    )

plt.xscale("log")
plt.yscale("log")
plt.xlabel("r")
plt.ylabel("Longitudinal correlation")
plt.title("Two-point longitudinal correlation")
plt.grid(True, which="both", alpha=0.3)
plt.legend(title="Lag", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)
plt.tight_layout()
plt.show()


Persist the correlation to NPZ for reuse later; reload to skip recomputation.

Quick look: longitudinal correlation vs separation for one representative lag (log-log).

Convert the two-point correlation into MSD (longitudinal/transverse).

In [ ]:
# Convert two-point correlations to one-point-like MSDs (analogous to msdd.m)
msd_from_2p = distinct_msd_from_two_point(
    corr,
    r_min=10,
    r_max=500,
    probe_radius=1.0,   # microns
    use_linear_fit=False,
)
mpl.style.use(files("trajkit.styles") / "plot.mplstyle")
t = msd_from_2p.dt
y = msd_from_2p.msd_longitudinal

mask = np.isfinite(t) & np.isfinite(y) & (t > 0)
t_fit = t[mask]
y_fit = y[mask]

slope = np.sum(t_fit * y_fit) / np.sum(t_fit ** 2)
D = slope / 2.0

t_line = np.linspace(t_fit.min(), t_fit.max(), 200)

plt.figure(figsize=(7, 5))
plt.scatter(t_fit, y_fit, s=30, color="C0", label="data")

plt.xscale("log")
plt.yscale("log")
plt.xlabel("lag time (s)")
plt.ylabel("MSD (longitudinal)")
plt.title("Two-point MSD (longitudinal)")
plt.grid(True, which="both", alpha=0.3)
plt.tight_layout()
plt.show()